In [1]:
# Save and load models
# Model progress can be saved during training. This means a model
# can resume where it left off and avoid long training times.
# When publishing research models and techniques, most machine
# learning practitioners share:

# 1. Code to create the model, and
# 2. The trained weigths, or parameters for the model

# Required to save models in HDF5 format
#pip install -q pyyaml h5py  

In [2]:
import os
import tensorflow as tf
from tensorflow import keras
print(tf.version.VERSION)

2.1.0


In [3]:
# Get an example dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0


In [4]:
# Define a model: Sart by building a simple sequential model
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Checkpoint callback usage

# Save checkpoints during training
# You can use a trained model without having to retrain it, or
# pick-up training where you left off. The tf.keras.callbacks.ModelChecpoint
# callback allows to continually save the model both during and at the end
# of the training

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 960/1000 [===========================>..] - ETA: 0s - loss: 1.1640 - accuracy: 0.6677
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 884us/sample - loss: 1.1329 - accuracy: 0.6790 - val_loss: 0.8032 - val_accuracy: 0.7320
Epoch 2/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.4456 - accuracy: 0.8681
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 249us/sample - loss: 0.4418 - accuracy: 0.8710 - val_loss: 0.5535 - val_accuracy: 0.8260
Epoch 3/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.3174 - accuracy: 0.9171
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 245us/sample - loss: 0.3037 - accuracy: 0.9210 - val_loss: 0.4695 - val_accuracy: 0.8530
Epoch 4/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.2270 - accuracy: 0.9435
Epoch

In [7]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 2.3501 - accuracy: 0.0780
Untrained model, accuracy:  7.80%


In [9]:
# Then load the weigths from the checkpoint and re-evaluate

# Loads the weigths
model.load_weights(checkpoint_path)

#Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose="2")
print("Restored model, acuracy:{:5.2f}%".format(100*acc))

Restored model, acuracy:86.50%


In [11]:
# Checkpoitn callbacks option

# The callback provides several options to provide unique names
# fro checkpoints and adjust the checkpoint frequency. Train
# anew model, and save uniquely named checkpoints once every five
# epochs

# Incluude the epoch in the file name (uses 'str.format')
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"

# Create a callback that saves the model's weight every 5 epochs

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5) # Indica que vamos a guardar los valores cada 5 peridos

# Create a new model instance
model = create_model()

# Save the weights using the checkpoint_path format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images,
         train_labels,
         epochs=50,
         callbacks=[cp_callback],
         validation_data=(test_images,test_labels),
         verbose=0)



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_1/cp.ckpt'

In [15]:
# Create a new model instance

model = create_model()

#Load the previously saved weigths
model.load_weights("training_2/cp-0050.ckpt")

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4829 - accuracy: 0.8770
Restored model, accuracy: 87.70%


In [16]:
# What are these files?
# The above code stores the weights to a collection of checkpoint
# formatted files that contain only the trained weights in a binary format
# Checkpoint contain:

# 1. One or more shards that contain your model's weights
# 2. An index file that indicates which weights are stored in which shard

# Manually save weights

# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("Restored model, accuracy : {:5.2f}%".format(100*acc))


1000/1000 - 0s - loss: 0.4829 - accuracy: 0.8770
Restored model, accuracy : 87.70%


In [17]:
# Save the entire model

# Call model.save to save a model's architecture, weigths and training configuration
# in a single file/folder. This allows you to export a model so it can be 
# used without access to orgininal Python code.

# Entire model can be saved in two different file formats (SavedModel and HDF5)

# SavedModel format

# The SavedModel format is another way to serialize models. Models
# saved in this format can be restored useing tf.keras.models.load_model

# Create and train a new model instance
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel
!mkdir -p saved_model
model.save('saved_model/my_model')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 554us/sample - loss: 1.1359 - accuracy: 0.6710
Epoch 2/5
1000/1000 [==============================] - 0s 123us/sample - loss: 0.4371 - accuracy: 0.8760
Epoch 3/5
1000/1000 [==============================] - 0s 130us/sample - loss: 0.2816 - accuracy: 0.9200
Epoch 4/5
1000/1000 [==============================] - 0s 124us/sample - loss: 0.2048 - accuracy: 0.9500
Epoch 5/5
1000/1000 [==============================] - 0s 124us/sample - loss: 0.1594 - accuracy: 0.9610
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [18]:
!ls saved_model/my_model

assets         saved_model.pb variables


In [20]:
# Reload a fresh keras model from the saved model
new_model = tf.keras.models.load_model('saved_model/my_model')

#check its architecture
new_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose= 2)
print("Restored model, accuracy : {:5.2f}%".format(100*acc))
print(new_model.predict(test_images).shape)

1000/1000 - 0s - loss: 0.4205 - accuracy: 0.8560
Restored model, accuracy : 85.60%
(1000, 10)


In [23]:
# HDF5 format
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file
# The '.h5' extension indicates that the model should be shaved to HDF5
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 693us/sample - loss: 1.1228 - accuracy: 0.6760
Epoch 2/5
1000/1000 [==============================] - 0s 151us/sample - loss: 0.4125 - accuracy: 0.8790
Epoch 3/5
1000/1000 [==============================] - 0s 173us/sample - loss: 0.2880 - accuracy: 0.9320
Epoch 4/5
1000/1000 [==============================] - 0s 138us/sample - loss: 0.2027 - accuracy: 0.9520
Epoch 5/5
1000/1000 [==============================] - 0s 145us/sample - loss: 0.1562 - accuracy: 0.9620


In [25]:
# Now recreate the model from that file:

# Recreate the exact same model, including its weigths and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the new model architecture
new_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [26]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 - 0s - loss: 0.4163 - accuracy: 0.8620
Restored model, accuracy: 86.20%


In [ ]:

# Saving custom objects:

# The key difference between HDF5 and SavedModel is that HDF5
# uses object configs to save the model architecture,
# while SavedModel saves the executino graph. 